<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/06_MOD17A3HGF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract annual NPP from MODIS

## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/LLBN")
aoi_name = "LLBN"

# GEE layer ID
layer_name = "RS_013"

# Date parameters
start_year = 2012
end_year = 2022

# Range doesn't include the stop value
year_list = ee.List(list(range(start_year, end_year+1)))

## MOD17A3HGF.061: Terra Net Primary Production Gap-Filled Yearly Global 500m



In [ ]:
modis = ee.ImageCollection("MODIS/061/MOD17A3HGF")

## NPP processing

In [ ]:
years = ee.List.sequence(start_year, end_year)

def yearly_NPP(focal_year):
  start = ee.Date.fromYMD(focal_year, 1, 1)
  end = ee.Date.fromYMD(focal_year, 12, 31)
  date_range = ee.DateRange(start, end)
  return modis \
        .select("Npp") \
        .filterDate(date_range) \
        .first() \
        .multiply(0.0001) \
        .set({"name": focal_year}) \
        .clip(aoi)


annual_image = years.map(yearly_NPP)
print(annual_image.getInfo())

## Check an element of list
year_mosaic  = ee.Image(annual_image.get(0))
label = ee.String(year_mosaic.get('name')).getInfo()
print(label)
print(year_mosaic.getInfo())


## Export data - create task

`filenamePrefix` should be in format: place_layer_timeperiod

In [ ]:
for i in  range(ee.List.length(annual_image).getInfo()):

  output_img =  ee.Image(annual_image.get(i))
  output_name = f"{layer_name}_{aoi_name}_{ee.String(output_img.get('name')).getInfo()}"

  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry().bounds(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 30,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
  task.start()
  print("STARTED TASK ", i+1)

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:ee.List.length(year_list).getInfo()]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)